In [1]:
import numpy as np
from exec_time import exec_time    # Home-made decorator to calculate the execution time of functions

## Finding-out the fastest exponentiation algorithm

#### Defining some helper functions

In [2]:
def exp_by_squaring(base, exp):
    """
    Homemade power function by squaring method based on well-known algorithm
    @wiki `https://en.wikipedia.org/wiki/Exponentiation_by_squaring`
    """
    if exp == 0:
        return 1
    if exp < 0:
        base = 1 / base
        exp = -exp
    res = 1
    while exp > 1:
        if exp % 2:
            res = res * base
            base = base * base
            exp = (exp - 1) / 2
        else:
            base = base * base
            exp = exp / 2
    return base * res


from itertools import compress
from functools import reduce
from operator import mul

def fast_exp(base, exp):
    """
    Calculate the exponentiation by applying reduce function to square the `base` according to bits of the `exponent`
    """
    def bits_of(m) :
        while m:
            yield m & 1
            m >>= 1

    def squares(b) :
        while True :
            yield b
            b *= b
    return reduce(mul, compress(squares(base), bits_of(exp)), 1)

In [3]:
@exec_time
def exec1(n):
    return np.power(n, n)

@exec_time
def exec2(n):
    return pow(n, n)

@exec_time
def exec3(n):
    return exp_by_squaring(n, n)

@exec_time
def exec4(n):
    return fast_exp(n, n)


n = 123
print("Start ...")
print("Calculating n^n using various functions and algorithms :", end="\n\n")

exec1(n)
exec2(n)
exec3(n)
exec4(n)

Start ...
Calculating n^n using various functions and algorithms :

*** Execution of exec1 :
> The result : 8741133280710329699
> Execution time : 0.028371810913085938 ms

*** Execution of exec2 :
> The result : 114374367934617190099880295228066276746218078451850229775887975052369504785666896446606568365201542169649974727730628842345343196581134895919942820874449837212099476648958359023796078549041949007807220625356526926729664064846685758382803707100766740220839267
> Execution time : 0.0064373016357421875 ms

*** Execution of exec3 :
> The result : 114374367934617190099880295228066276746218078451850229775887975052369504785666896446606568365201542169649974727730628842345343196581134895919942820874449837212099476648958359023796078549041949007807220625356526926729664064846685758382803707100766740220839267
> Execution time : 0.014066696166992188 ms

*** Execution of exec4 :
> The result : 11437436793461719009988029522806627674621807845185022977588797505236950478566689644660656836520154216

##### Results

- The built-in `pow` is the fastest solution. Furthermore, it gives the possibility to calculate the _modulo_ `n ^ m % p` directly and yields even faster results.

- We notice that Numpy's `power` gives wrong results for large integers due to the 64-bits width limitation.

## Extracting Digits

In [4]:
def f1(n):
    """
    Using built-in `pow` without the modulo argument
    """
    power = pow(n, n)
    units = power % 10
    tens = int((power % 100) / 10)
    return (units, tens)

def f2(n):
    """
    Using built-in `pow` with the modulo argument
    """
    power = pow(n, n, 100)
    units = power % 10
    tens = int((power / 10) % 10)
    return (units, tens)

def f3(n):
    """
    Using built-in `pow` with the modulo argument twice separately
    """
    units = pow(n, n, 10)
    tens = int(pow(n, n, 100) / 10)
    return (units, tens)

def f4(n):
    """
    Translate digits to characters and extract digits by index
    """
    power = str(pow(n, n))
    units = power[-1]
    tens = power[-2]
    return (units, tens)

In [21]:
@exec_time
def exec1(n):
    return f1(n)

@exec_time
def exec2(n):
    return f2(n)

@exec_time
def exec3(n):
    return f3(n)

@exec_time
def exec4(n):
    return f4(n)

In [22]:
n = 123

exec1(n)
exec2(n)
exec3(n)
exec4(n)

*** Execution of exec1 :
> The result : (7, 6)
> Execution time : 0.014543533325195312 ms

*** Execution of exec2 :
> The result : (7, 6)
> Execution time : 0.014781951904296875 ms

*** Execution of exec3 :
> The result : (7, 6)
> Execution time : 0.013828277587890625 ms

*** Execution of exec4 :
> The result : ('7', '6')
> Execution time : 0.020742416381835938 ms

